# SI Model with No Demographics

We will begin with perhaps the simplest possible model of an infectious disease - the SI model without demography.  This model has two classes of individuals, the infective and the susceptible, and 1 parameter, β, describing the per-timestep, per-infective person rate at which susceptibles transition to infectives.

$$
\dot{S} = -\frac{\beta*S*I}{N} \\

\dot{I} = \frac{\beta*S*I}{N}
$$

Noting that *S = N-I*, we can substitute *S* in the second equation to get a standard form logistic equation

$$
\dot{I} = \beta \frac{(I-N)I}{N} = \beta I (1 - \frac{I}{N})
$$

With solution 
$$
\frac{N}{1+(\frac{N}{I_0}-1)e^{-\beta t}}
$$


This notebook tests the implementation and behavior of the model as follows:
### Construct the model
In the first few cells, we do all the necessary imports.  Then we construct a single-patch LASER model with two components: Susceptibility and Transmission.  Finally, we initialize with a single infection and run.  The Susceptibility component endows both agents and patches in the model with a `susceptibility` quantity.  For the agents, this represents their state (infective = 0, susceptible = 1), and for the patch, it represents the total population of susceptible agents.  The differential equation above is converted into a stochastic finite difference equation

$$
\Delta I = Bin(S_t, 1-exp^{-\beta \Delta t \frac{I}{N}}) \\
S_{t+1} = S_t - \Delta I \\
I_{t+1} = I_t + \Delta I
$$


### Sanity check
The first test ensures certain basic constraints are being obeyed by the model.  We confirm that at each timestep, *S<sub>t</sub>=N<sub>t</sub>-I<sub>t</sub>*.  We also confirm that:

$$
I_t = I_0 + \sum_{t'=0}^t \Delta I_{t'}
$$  

### Scientific test
Finally, we come to the scientific test.  We select a few values of β, run the model, fit the outputs to the logistic equation, and compare the fitted value of β to the known value.  Because we are approximating continuously compounding growth, in the logistic equation, with a discrete time-step approximation, we expect the fitted values of β to be biased slightly downward - that is, the modeled trajectory is slightly slower than the continuous-time counterpart.  This error grows as β gets larger; the test fails if any of the fitted β values are more than 5% away from the known values.



## Constructing the model

In [ ]:
import numpy as np
import pandas as pd
from laser_core.propertyset import PropertySet
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize

from laser_generic import Model
from laser_generic.susceptibility import Susceptibility
from laser_generic.transmission import Transmission
from laser_generic.utils import seed_infections_randomly_SI

f"{np.__version__=}"

A note that order of the model components matters, because of how recording of relevant statistics into outputs occurs.  If transmission comes before susceptibility, then we have *N = S+I+ΔI*, because we record *I*, record *ΔI*, do transmission, then record *S* after the transmission process occurs.  With susceptibility first, we record *S*, then record *I*, then do transmission and record *ΔI*, and so we have *N = S+I*.

In [ ]:
pop = 1e6
init_inf = 1
scenario = pd.DataFrame(data=[["homenode", pop, "47°36′35″N 122°19′59″W"]], columns=["name", "population", "location"])
parameters = PropertySet({"seed": 6, "nticks": 730, "verbose": True, "beta": 0.05})
model = Model(scenario, parameters)


model.components = [
    Susceptibility,
    Transmission,
]

seed_infections_randomly_SI(model, ninfections=init_inf)
model.run()

## First set of sanity checks

In [ ]:
plt.plot(model.patches.cases.astype("int"), lw=4)
plt.plot(model.patches.populations[:-1] - model.patches.susceptibility.astype("int"), "--", lw=3)
plt.plot(init_inf + np.insert(np.cumsum(model.patches.incidence[:-1]).astype("int"), 0, 0), ":", lw=2, color="black")
plt.yscale("log")
plt.legend(["Population minus currently infected", "Susceptible", "Population minus cumulative infections (incidence)"])

print("S = N-I:  " + str(np.isclose(model.patches.susceptibility, model.patches.populations[:-1] - model.patches.cases).all()))
print(
    "I=sum(Delta I):  " + str(np.isclose(np.squeeze(model.patches.cases) - init_inf, [0, *np.cumsum(model.patches.incidence[:-1])]).all())
)

Introducing stochasticity into a model can result in more than just noise around a mean trajectory, and this simple model is a great example. In the early stages of the outbreak, the infection count is low, and the timing of those first few infections dictates when the logistic growth really takes off.  This appears as a random time-shift in the entire outbreak trajectory.    To illustrate this phenomenon, the below plot shows the model output, the expected logistic growth curve, and the expected logistic growth curve fit to the model with one free parameter -  an offset *t<sub>0</sub>*.  The resulting plot should show good concordance between the model output and the expected logistic equation with the known model inputs β and population.
The goodness of this fit could be turned into a strict pass/fail test down the line.

In [ ]:
def SI_logistic(t, beta, size, t0):
    return size / (1 + (size - 1) * np.exp(-beta * (t - t0)))


plt.plot(model.patches.cases, lw=4)

t = np.arange(model.params.nticks)


def objective(t0):
    return np.sum((1 - SI_logistic(t, model.params.beta, pop, t0) / np.squeeze(model.patches.cases.astype("int"))) ** 2)


result = minimize(objective, x0=10)
t0_opt = result.x[0]

plt.plot(t, SI_logistic(t, model.params.beta, pop, 0), "--", lw=2)
plt.plot(t, SI_logistic(t, model.params.beta, pop, t0_opt), "--", lw=3, color="k")
plt.yscale("log")
plt.legend(["Model output", "Logistic growth with known inputs, t0=0", f"Logistic growth with known inputs, best-fit t0 = {t0_opt:.1f}"])

## Scientific testing
Finally, we run the model for a range of β parameters, we freely fit the model output to the logistic equation, and we compare the known input parameters against the parameters fitted from output.  

Because we are approximating continuously compounding growth by discrete-time compounding growth, we should expect the fitted β to consistently be slightly underestimated relative to the true β, with the relative difference growing as β gets larger.   

In the future, we could probably compute the expected error from this approximation.  But for now, to make this a pass-fail test, we will raise a flag if the fitted β is more than 5% different than the known β.
 

In [ ]:
%%capture
nticks = 730
t = np.arange(730)
pop = 1e5

seeds = list(range(10))
betas = [0.01 * i for i in range(1, 11)]
output = pd.DataFrame(columns=["seed", "beta", "cases", "fitted_beta", "fitted_size", "fitted_t0"])
scenario = pd.DataFrame(data=[["homenode", pop, "47°36′35″N 122°19′59″W"]], columns=["name", "population", "location"])
for seed, beta in zip(seeds, betas):
    parameters = PropertySet({"seed": seed, "nticks": nticks, "verbose": True, "beta": beta})
    model = Model(scenario, parameters)
    model.components = [
        Susceptibility,
        Transmission,
    ]
    seed_infections_randomly_SI(model, ninfections=1)
    model.run()
    cases = [model.patches.cases[i][0] for i in range(nticks)]
    popt, pcov = curve_fit(SI_logistic, t, cases, p0=[0.05, 1.1e5, 1])

    output = pd.concat(
        [
            output,
            pd.DataFrame.from_dict(
                {
                    "seed": seed,
                    "beta": beta,
                    "cases": [np.array(cases)],
                    "fitted_beta": popt[0],
                    "fitted_size": popt[1],
                    "fitted_t0": popt[2],
                }
            ),
        ],
        ignore_index=True,
    )

In [ ]:
plt.plot(output["beta"], output["fitted_beta"], "o")
plt.xlabel("True beta")
plt.ylabel("Fitted beta")
plt.figure()
plt.plot(output["beta"], (output["beta"] - output["fitted_beta"]) / output["beta"], "o")
plt.xlabel("True beta")
plt.ylabel("Relative error in fitted beta")

In [ ]:
print(
    "All fitted beta are within 5% of known beta: " + str(np.all(np.abs((output["beta"] - output["fitted_beta"]) / output["beta"]) < 0.05))
)